In [1]:
import requests
import numpy as np
import time
import pandas as pd
import os

In [2]:
API_KEY = os.environ.get("RIOT_API_KEY")

In [3]:
def get_summoner_ids(region: str):
    """
    Gets random summoner ids from North America from the various ranks (Diamond:Platinum:Gold:Silver:Bronze)
    
    Returns a dictionary by rank of summoner ids
    """
    ranks = ['DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'BRONZE']
    tiers = ['I', 'II', 'III', 'IV']
    summoner_ids_rank = dict()
    for rank in ranks:
        summoner_ids = set()
        for tier in tiers:
            for page_num in range(1, 4+1):
                URL = f'https://{region}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{tier}/?page={page_num}&api_key={API_KEY}'
                response = requests.get(URL)
                summoner_ids.update([s['summonerId'] for s in response.json()])
                time.sleep(1.5)
        summoner_ids_rank[rank] = summoner_ids
    return summoner_ids_rank

In [78]:
na1_summoner_ids = get_summoner_ids('na1')

In [13]:
euw1_summoner_ids = get_summoner_ids('euw1')

In [4]:
kr_summoner_ids = get_summoner_ids('kr')

In [5]:
def get_champion_masteries(summoner_id_rank: dict, region: str):
    """
    Gets the champion mastery information for each summoner id by rank in the given regio
    
    Returns a dictionary by rank containing the list of each players top 5 champions
    """
    min_champs = 5
    min_exp = 21000
    
    print(region)
    champion_masteries = dict()
    
    for rank in summoner_id_rank:
        champions_by_rank = []
        for i, summoner_id in enumerate(summoner_id_rank[rank]):
            champs = []
            URL = f'https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{summoner_id}?api_key={API_KEY}'
            response = requests.get(URL)
            champion_list = response.json()
            
            if len(champion_list) < min_champs:
                continue
            
            if champion_list[4]['championPoints'] < min_exp:
                continue
            
            champs = [champion_list[i]['championId'] for i in range(5)]
            champions_by_rank.append(champs)
            time.sleep(1.2)
            if i % 50 == 0:
                print(f"finished summoner id at index {i}")
        champion_masteries[rank] = champions_by_rank
        print(f'Finished processing rank: {rank}')
        time.sleep(60)
    return champion_masteries

In [124]:
na1_top5_champs = get_champion_masteries(na1_summoner_ids, 'na1')

na1
finished summoner id at index 50
finished summoner id at index 400
finished summoner id at index 700
finished summoner id at index 900
finished summoner id at index 1200
finished summoner id at index 1400
finished summoner id at index 1600
finished summoner id at index 1750
finished summoner id at index 1800
finished summoner id at index 2050
finished summoner id at index 2250
finished summoner id at index 2750
finished summoner id at index 2950
finished summoner id at index 3000
finished summoner id at index 3200
Finished processing rank: DIAMOND
finished summoner id at index 50
finished summoner id at index 100
finished summoner id at index 250
finished summoner id at index 300
finished summoner id at index 400
finished summoner id at index 550
finished summoner id at index 700
finished summoner id at index 750
finished summoner id at index 900
finished summoner id at index 1200
finished summoner id at index 1250
finished summoner id at index 1550
finished summoner id at index 16

In [128]:
na1_top5_champs['DIAMOND'][1]

[84, 147, 157, 235, 62]

In [19]:
columns = ['Champion1', 'Champion2', 'Champion3', 'Champion4', 'Champion5']

In [130]:
na_1_diamond = pd.DataFrame(na1_top5_champs['DIAMOND'], columns=columns)

In [131]:
na_1_diamond.head()

,Champion1,Champion2,Champion3,Champion4,Champion5
0,145,8,555,81,67
1,84,147,157,235,62
2,76,64,104,121,131
3,432,25,38,30,79
4,29,555,350,235,267


In [135]:
for rank in na1_top5_champs:
    na_1_new_df = pd.DataFrame(na1_top5_champs[rank], columns=columns)
    na_1_new_df.to_csv(f'data/na_{rank.lower()}.csv', index=False)

In [16]:
euw1_top5_champs = get_champion_masteries(euw1_summoner_ids, 'euw1')

euw1
finished summoner id at index 0
finished summoner id at index 50
finished summoner id at index 100
finished summoner id at index 150
finished summoner id at index 200
finished summoner id at index 250
finished summoner id at index 300
finished summoner id at index 350
finished summoner id at index 400
finished summoner id at index 450
finished summoner id at index 500
finished summoner id at index 550
finished summoner id at index 600
finished summoner id at index 650
finished summoner id at index 700
finished summoner id at index 750
finished summoner id at index 800
finished summoner id at index 850
finished summoner id at index 900
finished summoner id at index 950
finished summoner id at index 1000
finished summoner id at index 1050
finished summoner id at index 1100
finished summoner id at index 1150
finished summoner id at index 1200
finished summoner id at index 1250
finished summoner id at index 1300
finished summoner id at index 1350
finished summoner id at index 1400
fin

In [22]:
for rank in euw1_top5_champs:
    euw_new_df = pd.DataFrame(euw1_top5_champs[rank], columns=columns)
    euw_new_df.to_csv(f'data/euw_{rank.lower()}.csv', index=False)

In [8]:
len(kr_summoner_ids['BRONZE'])

3220

In [ ]:
kr_summoner_ids

In [9]:
kr_top5_champs = get_champion_masteries(kr_summoner_ids, 'kr')

kr
finished summoner id at index 200
finished summoner id at index 250
finished summoner id at index 350
finished summoner id at index 400
finished summoner id at index 450
finished summoner id at index 600
finished summoner id at index 650
finished summoner id at index 700
finished summoner id at index 750
finished summoner id at index 800
finished summoner id at index 850
finished summoner id at index 900
finished summoner id at index 950
finished summoner id at index 1000
finished summoner id at index 1050
finished summoner id at index 1100
finished summoner id at index 1200
finished summoner id at index 1250
finished summoner id at index 1300
finished summoner id at index 1350
finished summoner id at index 1400
finished summoner id at index 1450
finished summoner id at index 1500
finished summoner id at index 1600
finished summoner id at index 1700
finished summoner id at index 1750
finished summoner id at index 1800
finished summoner id at index 1850
finished summoner id at index 

In [20]:
for rank in kr_top5_champs:
    kr_new_df = pd.DataFrame(kr_top5_champs[rank], columns=columns)
    kr_new_df.to_csv(f'data/kr_{rank.lower()}.csv', index=False)